# Import libraries

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt 
plt.style.use('fivethirtyeight') 
from wordcloud import WordCloud
import enchant
import re 

In [2]:
df = pd.read_csv("cleaned-dataset.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)

df.head()

,the,to,ect,and,for,of,a,you,hou,in,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Label
0,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8,13,24,6,6,2,102,1,27,18,...,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,8,0,0,4,...,0,0,0,0,0,0,0,0,0,0
3,0,5,22,0,5,1,51,2,10,1,...,0,0,0,0,0,0,0,0,0,0
4,7,6,17,1,5,2,57,0,9,3,...,0,0,0,0,0,0,0,1,0,0


# Exploration data analysis

## 1. Correlation between words and email labels

In [ ]:
# Calculate correlation coefficients between each word and the target variable
correlations = df.corr()['Label'].sort_values()

In [ ]:
# Top 10 words most strongly associated with spam emails
top_spam = correlations[1:].tail(10)

print("Top 10 words most strongly associated with spam emails:")
print(top_spam)

# Top 10 words most strongly associated with not-spam emails
top_not_spam = correlations[1:].head(10)
print("Top 10 words most strongly associated with not-spam emails:")
print(top_not_spam)

The top 10 words most strongly associated with spam emails, in order of highest correlation coefficient, are: more, our, able, best, ur, sex, sec, money, and soft. The strongest word association with the spam label is the word "more", followed closely by "our" and "able". These words may be common in spam emails because they are often used to make false claims or offer something of value in exchange for personal information or money.

On the other hand, the top 10 words most strongly associated with not-spam emails, in order of highest negative correlation coefficient, are: hpl, hanks, thank, attached, daren, forwarded, subject, hp, aren, and nom. The strongest word association with the not-spam label is the word "hpl", followed by "hanks" and "thank". These words may be common in not-spam emails because they are often used in professional or formal communications.

In [ ]:
# Create separate dataframes for spam and non-spam emails
spam_df = df[df['Label'] == 1][top_spam.index]
not_spam_df = df[df['Label'] == 0][top_not_spam.index]

# Calculate correlation matrices for spam and non-spam emails
spam_corr = spam_df.corr()
not_spam_corr = not_spam_df.corr()

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(20, 8))
sb.heatmap(spam_corr, cmap="coolwarm", annot=True, fmt='.2f', ax=ax1)
ax1.set_title("Correlation Matrix for Top 10 Words in Spam Emails")
sb.heatmap(not_spam_corr, cmap="coolwarm", annot=True, fmt='.2f', ax=ax2)
ax2.set_title("Correlation Matrix for Top 10 Words in Non-Spam Emails")
plt.show()

Analyzing the correlation matrices for the top 10 words in spam and non-spam emails, we can observe some interesting patterns.

In the spam email matrix, we can see that some words are highly correlated. For instance, `our` and `ur` have a correlation coefficient of 0.76, followed by `ur` and `sec` with 0.67. Additionally, `able` and `ur` have a correlation of 0.50, and `money` and `our` have a correlation of 0.40. Surprisingly, `sex` has almost no correlation with other words except for a slight negative correlation with `money` and `sec`.

On the other hand, the non-spam email matrix shows much higher correlations between words. Some of these correlations are expected, such as `hp` and `hpl` with a coefficient of 0.99, `hanks` and `thanks` with 0.83, and `forwarded` and `subject` with 0.73. These findings indicate that non-spam emails tend to use words in a more structured and cohesive manner compared to spam emails.

##  2. Distribution of label classes

In [ ]:
spam_count = (df['Label'] == 1).sum()
not_spam_count = (df['Label'] == 0).sum()

plt.bar(['Spam', 'Not Spam'], [spam_count, not_spam_count])
plt.title('Distribution of email labels')
for i, v in enumerate([spam_count, not_spam_count]):
    plt.text(i, v, str(v), color='black', ha='center')
plt.show()

In this dataset, there is 3672 non-spam emaild and 1500 spam emails. This indicates that the dataset is imbalanced because the number of non-spam emails is significantly higher than the number of spam emails. Therefore, the model may have better performance on non-spam emails but may not perform well on spam emails.Furthermore,an imbalanced data may lead to more incorrect predictions as the model may have a bias towards the majority class and may not be able to capture the patterns and relationships in the minority class.

In the end, we have used several techniques to solve this problem. 

## 3. Word frequency in spam and not-spam emails

In [ ]:
spam_words = df.loc[df['Label'] == 1].iloc[:, 1:-1].sum().sort_values(ascending=False)[:10]
print("\nTop 10 words with highest frequency in spam emails:")
for word, count in spam_words.items():
    print(f"{word}: {count}")

not_spam_words = df.loc[df['Label'] == 0].iloc[:, 1:-1].sum().sort_values(ascending=False)[:10]
print("\nTop 10 words with highest frequency in non-spam emails:")
for word, count in not_spam_words.items():
    print(f"{word}: {count}")


The frequency of the top 10 most common words in both spam and not-spam emails were also calculated. The results show that the most common words in both types of emails are similar, with "e", "t", "a", "o", "n", "r", "i", "s", "l", and "c" being the top 10 in both cases, though their frequencies are higher in not-spam emails.

It is worth noting that while the frequency of these common words may not be particularly informative in distinguishing between spam and not-spam emails on their own, they could be useful in combination with other features in a machine learning model. For example, the presence or absence of these common words could be used in combination with the top 10 words most strongly associated with either label to improve the accuracy of email classification models.

 Instead of just letters,lets try to words with the highest frequency in spam and non spam emails to have a more meaningful visualisation of the data

In [ ]:

pattern = r'^[A-Za-z]{3,}$' # Regular expression pattern for single letter column names
filtered_df = df.filter(regex=pattern)
d = enchant.Dict("en_US") # Create an instance of the PyEnchant English dictionary
filtered_words = filtered_df.columns[1:-1].tolist() # Convert the filtered columns to a list
filtered_words = [word for word in filtered_words if len(word) > 3 or d.check(word)] # Filter out words of length 3 that are not English words
filtered_df = df[['Label'] + filtered_words]
spam_words = filtered_df.loc[df['Label'] == 1].iloc[:, 1:-1].sum().sort_values(ascending=False)[:10]
print("\nTop 100 words with highest frequency in spam emails:")
for word, count in spam_words.items():
    print(f"{word}: {count}")


In [ ]:
not_spam_words = filtered_df.loc[df['Label'] == 0].iloc[:, 1:-1].sum().sort_values(ascending=False)[:10]
print("\nTop 100 words with highest frequency in non-spam emails:")
for word, count in not_spam_words.items():
    print(f"{word}: {count}")

    

In [ ]:
# Find the unique words in each list
unique_spam_words = set(spam_words.index) - set(not_spam_words.index)
unique_not_spam_words = set(not_spam_words.index) - set(spam_words.index)
plt.figure(figsize=(10,10))
plt.barh(list(unique_spam_words), [spam_words[word] for word in unique_spam_words])
plt.title("Unique words in spam emails")
plt.xlabel("Frequency")
plt.show()


In [ ]:
#Horizontal bar chart to visualize the unique words in non-spam emails
plt.figure(figsize=(10,10))
plt.barh(list(unique_not_spam_words), [not_spam_words[word] for word in unique_not_spam_words])
plt.title("Unique words in non-spam emails")
plt.xlabel("Frequency")
plt.show()

## 4. Distribution of word frequency

In [ ]:
# Calculate the frequency of each word
word_counts_unfiltered = df.iloc[:, :-1].sum().sort_values(ascending=False)
top_10 = word_counts_unfiltered.head(10)
for word, count in top_10.items():
    print(f"{word}: {count}")
# Create a histogram of the word frequencies
plt.figure(figsize=(12, 6))
sb.histplot(word_counts_unfiltered, log_scale=True, bins=50)
plt.xlabel("Word Frequency")
plt.ylabel("Count")
plt.title("Distribution of Word Frequency Unfiltered")

plt.show()

In [ ]:
word_counts_filtered = filtered_df.iloc[:, :-1].sum().sort_values(ascending=False)
top_10 = word_counts_filtered.head(10)
for word, count in top_10.items():
    print(f"{word}: {count}")
# Create a histogram of the word frequencies
plt.figure(figsize=(12, 6))
sb.histplot(word_counts_filtered, log_scale=True, bins=50)
plt.xlabel("Word Frequency")
plt.ylabel("Count")
plt.title("Distribution of Word Frequency Filtered")
plt.show()

## 5. Distribution of Email Length 

In [ ]:
email_lengths = df.iloc[:, :-1].sum(axis=1)

plt.figure(figsize=(12, 6))
sb.kdeplot(email_lengths, shade=True)
plt.xlabel("Email Length")
plt.title("Distribution of Email Length")
plt.xlim(0, 15000)

# Find the peak of the density plot
density = sb.kdeplot(email_lengths).get_lines()[0].get_data()
x_peak = density[0][density[1].argmax()]
y_peak = density[1].max()

# Add a vertical line to the plot to indicate the peak
plt.axvline(x_peak, color='red', linestyle='--')
plt.text(x_peak+100, y_peak-0.00004, f"Peak: {int(x_peak)}", color='red', fontsize=12)

plt.show()

Based on the peak at 302, it appears that a significant number of emails in the dataset have a length of around 300 words. This could suggest that many emails in the dataset are concise and to the point, with fewer unnecessary or extraneous words. Additionally, the fact that the density plot has a long tail towards the right indicates that there are some emails in the dataset that are significantly longer than the majority. This could indicate that there are certain emails in the dataset that are more complex or detailed, and may require more attention or analysis. Overall, the distribution of email length suggests that there is a wide range of variability in email length in the dataset, with many emails being relatively short and concise, but with some longer and more complex emails as well.

## 6. Distribution of Email Length by Label

In [ ]:
spam_lengths = df[df['Label'] == 1].iloc[:, :-1].sum(axis=1)
not_spam_lengths = df[df['Label'] == 0].iloc[:, :-1].sum(axis=1)

# Calculate peak for each distribution
spam_peak = spam_lengths.value_counts().idxmax()
not_spam_peak = not_spam_lengths.value_counts().idxmax()

# Print peaks
print("Peak of spam email length distribution: ", spam_peak)
print("Peak of not-spam email length distribution: ", not_spam_peak)

# Density plot
plt.figure(figsize=(12, 6))
sb.kdeplot(spam_lengths, shade=True, color='red', label='Spam')
sb.kdeplot(not_spam_lengths, shade=True, color='blue', label='Not-Spam')

# Add vertical lines for peaks
plt.axvline(spam_peak, color='red', linestyle='--', ymax=0.05)
plt.axvline(not_spam_peak, color='blue', linestyle='--', ymax=0.05)
plt.axvline(x=10000, color='black', linestyle='--')

plt.xlim(-1000,30000)
plt.xlabel("Email Length")
plt.title("Distribution of Email Length by Label")
plt.annotate('Spam', xy=(6000, 0.00008), color='red')
plt.annotate('Not-Spam', xy=(6000, 0.0002), color='blue')
plt.legend()
plt.show()

From the plot, we can see that the distributions of email length for spam and non-spam emails have some overlap, but there are also some noticeable differences. 
Non-spam emails generally have a higher peak and a shorter tail. In contrast, spam emails have a longer length, with a lower peak and a longer tail.
This tells me that if email lengths are more than 10000 words,there is a higher chance that it is a spam email.

We can also see that the number of non-spam emails is much higher than the number of spam emails, as indicated by the larger area under the non-spam curve. This is not surprising, as non-spam emails are typically more common in our inboxes.

Overall, this plot suggests that email length could be a useful feature for distinguishing between spam and non-spam emails, as there are clear differences in the distribution of email lengths between the two categories.

In [ ]:
spam_lengths = df[df['Label'] == 1].iloc[:, :-1].sum(axis=1)
not_spam_lengths = df[df['Label'] == 0].iloc[:, :-1].sum(axis=1)

# Calculate peak for each distribution
spam_peak = spam_lengths.value_counts().idxmax()
not_spam_peak = not_spam_lengths.value_counts().idxmax()

# Density plot
plt.figure(figsize=(12, 6))
sb.kdeplot(spam_lengths, shade=True, color='red', label='Spam')
sb.kdeplot(not_spam_lengths, shade=True, color='blue', label='Not-Spam')

# Add vertical lines for peaks
plt.axvline(spam_peak, color='red', linestyle='--', ymax=0.05)
plt.axvline(not_spam_peak, color='blue', linestyle='--', ymax=0.05)
plt.axvline(x=10000, color='black', linestyle='--')

plt.xlim(10000,30000)
plt.ylim(0,0.00005)
plt.xlabel("Email Length")
plt.title("Distribution of Email Length by Label\n Zoomed in to see length more than 10000")
plt.annotate('Spam', xy=(6000, 0.00008), color='red')
plt.annotate('Not-Spam', xy=(6000, 0.0002), color='blue')
plt.legend()
plt.show()

## 7. Distribution of Email Length by Word Frequency Quartiles

In [ ]:
df_copy = df
df_copy['EmailLength'] = df.iloc[:, :-1].sum(axis=1)
# Compute the total word frequency for each email
word_freq = df.iloc[:, :-1].sum(axis=1)

# Divide the emails into quartiles based on word frequency
quartiles = pd.qcut(word_freq, q=4, labels=["Q1", "Q2", "Q3", "Q4"])

# Create a new column in the DataFrame to store the quartile information
df["WordFreqQuartile"] = quartiles

# Plot the distribution of email lengths within each quartile
plt.figure(figsize=(12, 6))
sb.kdeplot(data=df, x="EmailLength", hue="WordFreqQuartile")
plt.xlabel("Email Length")
plt.title("Distribution of Email Length by Word Frequency Quartiles")
plt.xlim(0, 5000)
plt.show()

 From the graph, we can see that the emails with the highest word frequency tend to have a longer length compared to those with lower word frequency.

Additionally, we can see that the distribution of email lengths becomes wider and more spread out as the word frequency increases. This suggests that emails with higher word frequency are more likely to have a wider range of lengths.

In [ ]:
# Extract words from spam emails
spam_words = df.loc[df['Label'] == 1].iloc[:, 1:-1].sum().sort_values(ascending=False)[:50]
spam_text = ' '.join([word for word in list(spam_words.index) if len(word) > 1 and not re.match(r'^\W*$', word) and not re.match(r'^\d*$', word) and word != 'EmailLength'])

# Generate word cloud for spam emails
spam_wordcloud = WordCloud(width=800, height=800, background_color='white').generate(spam_text)

# Extract words from non-spam emails
not_spam_words = df.loc[df['Label'] == 0].iloc[:, 1:-1].sum().sort_values(ascending=False)[:50]
not_spam_text = ' '.join([word for word in list(not_spam_words.index) if len(word) > 1 and not re.match(r'^\W*$', word) and not re.match(r'^\d*$', word) and word != 'EmailLength'])

# Generate word cloud for non-spam emails
not_spam_wordcloud = WordCloud(width=800, height=800, background_color='white').generate(not_spam_text)

# Plot the word clouds
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs[0].imshow(spam_wordcloud, interpolation='bilinear')
axs[0].set_title('Word Cloud for Spam Emails')
axs[0].axis('off')
axs[1].imshow(not_spam_wordcloud, interpolation='bilinear')
axs[1].set_title('Word Cloud for Non-Spam Emails')
axs[1].axis('off')
plt.show()

It is hard to get a meaningful insight of the wordcloud at a glance,hence we use the filtered dataframe and a smaller range of words.

In [ ]:
# Extract words from spam emails
spam_words = filtered_df.loc[df['Label'] == 1].iloc[:, :-1].sum().sort_values(ascending=False)[:30]
spam_text = ' '.join(list(spam_words.index))
unique_spam_words = set(spam_words.index) - set(not_spam_words.index)

# Add unique words to the spam text
unique_spam_text = ' '.join(list(unique_spam_words))
spam_text += f" {unique_spam_text}"

# Generate word cloud for spam emails
spam_wordcloud = WordCloud(width=800, height=800, background_color='white').generate(spam_text)

# Extract words from non-spam emails
not_spam_words = filtered_df.loc[df['Label'] == 0].iloc[:, :-1].sum().sort_values(ascending=False)[:30]
not_spam_text = ' '.join(list(not_spam_words.index))
unique_not_spam_words = set(not_spam_words.index) - set(spam_words.index)

# Add unique words to the non-spam text
unique_not_spam_text = ' '.join(list(unique_not_spam_words))
not_spam_text += f" {unique_not_spam_text}"

# Generate word cloud for non-spam emails
not_spam_wordcloud = WordCloud(width=800, height=800, background_color='white').generate(not_spam_text)

# Plot the word clouds
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs[0].imshow(spam_wordcloud, interpolation='bilinear')
axs[0].set_title('Word Cloud for Spam Emails')
axs[0].axis('off')
axs[1].imshow(not_spam_wordcloud, interpolation='bilinear')
axs[1].set_title('Word Cloud for Non-Spam Emails')
axs[1].axis('off')
plt.show()


# Identify outlier words


So far what we have been trying to visualise and see how to better identify whether an email is spam using its length and the occurence of certain words in the email.Though not as an important factor as finding the overall pattern of the frequency of words,we thought it would be good to visualise the outliers as outliers may skew statistical analysis,thus affecting the accuracy of our predictive models, and result in incorrect conclusions being drawn from the data. 

In [ ]:
data = pd.Series(word_counts_filtered)
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1
maximum=word_counts_filtered.max()
minimum = word_counts_filtered.min()
print("Q1:", Q1)
print("Q3:", Q3)
print("IQR:", IQR)
print("Min:",minimum)
print("Max:",maximum)

In [ ]:
plt.figure(figsize=(12,6))
sb.boxplot(data=word_counts_filtered, orient="h", palette="Set2")
plt.title("Distribution of Word Frequency filtered")
plt.xlabel("Frequency")

# Set x-axis limits to improve visualization
plt.xlim(right=maximum)

plt.show()


In [ ]:
rule = ((data < (Q1 - 1.5 * (Q3 - Q1))) | (data > (Q3 + 1.5 * (Q3 - Q1))))
print("Number of outliers:", rule.sum())

In [ ]:
word_counts_filtered = filtered_df.iloc[:, :-1].sum().sort_values(ascending=False)
top_10 = word_counts_filtered.head(10)
for word, count in top_10.items():
    print(f"{word}: {count}")

# Conclusion

Overall,one thing that is pretty apparent when doing our data visualisation is the frequency of stop words,which are common words that are unlikely to be informative, such as `this`, `and`, and `for`.Removing these words from the data can help to reduce the dimensionality of the data and improve the algorithm's ability to learn meaningful patterns in the word frequencies.Single letters will should also be removed(will be done in data resampling)